In [ ]:
!pip install --upgrade pip
!pip install openai
!pip install tiktoken

In [ ]:
!pip install -U boto3 botocore

In [ ]:
!pip install transformers==4.30.2
!pip install accelerate -U
!pip install sentencepiece -U

# 加载embedding 模型的tokenizer，用于计算token

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/paraphrase-multilingual-mpnet-base-v2')

# Option1: 用本地Sagemaker模型进行增强

In [588]:
import json
import boto3

smr_client = boto3.client("sagemaker-runtime")

parameters = {
  "max_length": 4096,
  "top_p":1,
    "temperature":0.1
}

In [589]:
import io
class StreamScanner:
    """
    A helper class for parsing the InvokeEndpointWithResponseStream event stream. 
    
    The output of the model will be in the following format:
    ```
    b'{"outputs": [" a"]}\n'
    b'{"outputs": [" challenging"]}\n'
    b'{"outputs": [" problem"]}\n'
    ...
    ```
    
    While usually each PayloadPart event from the event stream will contain a byte array 
    with a full json, this is not guaranteed and some of the json objects may be split across
    PayloadPart events. For example:
    ```
    {'PayloadPart': {'Bytes': b'{"outputs": '}}
    {'PayloadPart': {'Bytes': b'[" problem"]}\n'}}
    ```
    
    This class accounts for this by concatenating bytes written via the 'write' function
    and then exposing a method which will return lines (ending with a '\n' character) within
    the buffer via the 'readlines' function. It maintains the position of the last read 
    position to ensure that previous bytes are not exposed again. 
    """
    
    def __init__(self):
        self.buff = io.BytesIO()
        self.read_pos = 0
        
    def write(self, content):
        self.buff.seek(0, io.SEEK_END)
        self.buff.write(content)
        
    def readlines(self):
        self.buff.seek(self.read_pos)
        for line in self.buff.readlines():
            if line[-1] != b'\n':
                self.read_pos += len(line)
                yield line[:-1]
                
    def reset(self):
        self.read_pos = 0

### 需要填入对于的sagemaker endpoint

In [591]:
endpoint_name='qwen-stream-int4-2023-08-31-02-25-35-582-endpoint'

def call_llm(prompt,endpoint_name=endpoint_name,parameters=parameters):
    response_model = smr_client.invoke_endpoint_with_response_stream(
            EndpointName=endpoint_name,
            Body=json.dumps(
            {
                "inputs": prompt,
                "parameters": parameters,
                "history" : [],
                "stream":True
            }
            ),
            ContentType="application/json",
        )

    event_stream = response_model['Body']
    scanner = StreamScanner()
    text = ''
    for event in event_stream:
        scanner.write(event['PayloadPart']['Bytes'])
        for line in scanner.readlines():
            try:
                resp = json.loads(line)
                print(resp.get("outputs")['outputs'], end='')
                text +=resp.get("outputs")['outputs']
            except Exception as e:
                # print(line)
                continue
    return text

# Option 2: 用Chatgpt3.5，添加api_key

In [ ]:
import os,json
import openai
openai.api_key = ''


def call_llm(prompt):
    completion = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-16k-0613",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message['content']


# 创建结果格式化解析（本地部署模型，输出结果的格式不太稳定，需要格式化，如果使用ChatGPT不需要这个解析）

In [594]:
import re
##example cases
#line1 = '问题1:我在游戏中怎么查看当前升级所需要的经验？ 回答1: 15级解锁神奇宝玩家可以点击左上角查看当前升级所需要的经验。'
#line2 = '问题1:我在游戏中怎么查看当前升级所需要的经验？'
#line3 = '回答1: 充值可以让玩家获得更多的游戏内物品和特权，如游戏道具、游戏时间等。它们可以帮助玩家更快地提高游戏水平和游戏体验。'
#line4 = '问题1: 12我在游戏中遇到了其他玩家的不当言论，应该怎样举报？ 回答1: 如果在游戏中遇到其他玩家不当言论的情况，可以在聊天框点击对方的头像选择举报或者联系客服提供聊天截图进行举报（建议同时点击对方头像选择屏蔽）。 问题2: 举报需要提供哪些信息？ 回答2: 举报时需要提供相关的聊天截图以及被举报者的账号信息，以确保举报的有效性。希望以上回答能够帮助您更好地了解如何举报。'
pattern_1 = re.compile(r'(问题\d*:[^回答:]*)(回答\d*:[^问题:]*)',re.I)
pattern_2 = re.compile(r'问题\d*:[^回答:]*',re.I)
pattern_3 = re.compile(r'回答\d*:[^问题:].*',re.I)

def parse_response(response):
    new_data = []
    qna = []
    for line in response.split('\n'):
        
        groups = pattern_1.findall(line)
        for question,answer in groups: ##如果问题和答案在一行
            question = re.sub(r'^问题\d*:','',question)
            answer = re.sub(r'^回答\d*:','',answer)
            new_data.append((question,answer))
            
        if not len(groups): ##如果问题和答案是分行
            groups_q = pattern_2.match(line)
            groups_a = pattern_3.match(line)
            if groups_q:
                question = re.sub(r'^问题\d*:','',groups_q[0])
                qna.append(question)
            elif groups_a:
                answer = re.sub(r'^回答\d*:','',groups_a[0])
                qna.append(answer)
            if len(qna) == 2:
                new_data.append((qna[0],qna[1]))
                qna = []
                
    return new_data

# 读取原始FAQ文件

In [597]:
origin_faqfilename = "topwar_faq/topwarfaq230908.faq"

In [598]:
import os
def parse_faq(file_content,QA_SEP='====='):
    arr = file_content.split(QA_SEP)
    list_arr = []
    for item in arr:
        # print(item)
        try:
            question, answer = item.strip().split("\n", 1)
            question = question.replace("Question: ", "")
            answer = answer.replace("Answer: ", "")
            list_arr.append((answer.strip(),question.strip()))
        except:
            pass
    return list_arr


all_data = []
with open(origin_faqfilename) as f:
        data = f.read()
        all_data += parse_faq(data)
        
print(f"data size:{len(all_data)}")

data size:2178


####  如果用chat gpt，可以直接使用以下模板
```python
prompt = f"""
如下三个反引号中是《口袋奇兵》游戏的相关知识信息, 请基于这部分知识信息生成{nums}的问题以及对应答案
```
{context}
```
要求尽可能详细全面, 并且遵循如下规则:
1. 生成的内容不要超出反引号中信息的范围
2. 问题部分需要以"Question:"开始
3. 答案部分需要以"Answer:"开始
"""
```

In [616]:
import math,time

newfaq_filename = 'docs/topwar_enrich_faq_0911.faq'

def generate_query(new_data,alldata,tokenizer):
    faqtext = ''
    for i, (answer, query) in enumerate (alldata):
        context = f"问:{query}\n答:{answer}"
        tokens = len(tokenizer(context)['input_ids'])
        nums =  math.ceil(tokens/40)
        
        ##如果用chat gpt，可以直接使用以下模板
        # prompt = f"""
        # 如下三个反引号中是《口袋奇兵》游戏的相关知识信息, 请基于这部分知识信息生成{nums}的问题以及对应答案
        # ```
        # {context}
        # ```
        # 要求尽可能详细全面, 并且遵循如下规则:
        # 1. 生成的内容不要超出反引号中信息的范围
        # 2. 问题部分需要以"Question:"开始
        # 3. 答案部分需要以"Answer:"开始
        # """
        prompt= f"请根据以下内容生成{nums}个独立的新问题和对应问题的回答\n ```\n{context}\n```，问题和回答使用一对一的形式展示,问题的语气模拟你是一个游戏玩家，例如:\n问题1: \n回答1:\n\n问题2:\n回答2:\n\n"
        print (f"\n-------[{i}]-----Origin FAQ--------\n")
        print (context)
        print (f"\n-------[{i}]-----New FAQ--------\n")
        response = call_llm_sagemaker(prompt)
        parsed_qa = parse_response(response)
        
        for q,a in parsed_qa:
            faqtext+=f"Question: {q}\nAnswer: {a}\n=====\n"
        with open(newfaq_filename,"w") as f:
            f.write(faqtext)

#### 开始合成faq

In [ ]:
generate_query(newdata,all_data,tokenizer)


-------[0]-----Origin FAQ--------

问:小游戏在哪
答:目前只有10-20级玩家可以看到小游戏中心，您可以点击右上角的（大喇叭）常规活动，翻到最后面就可以看到了哦！

-------[0]-----New FAQ--------

问题1：小游戏在哪里可以找到？ 回答1：目前只有10-20级玩家可以看到小游戏中心。您可以点击右上角的（大喇叭）常规活动，翻到最后面就可以看到了哦！ 问题2：小游戏中心有哪些游戏可以玩呢？ 回答2：小游戏中心提供了多种不同种类的游戏供玩家畅玩，包括简单的休闲游戏、策略游戏等等。每个游戏的玩法和规则都不相同，您可以根据自己的兴趣爱好来选择适合自己的游戏。
-------[1]-----Origin FAQ--------

问:装饰物
答:装饰物是一种游戏道具，可以通过摆放在基地，获得属性增益
1.15级解锁神奇宝箱建筑，每隔一定时间会获得装饰宝箱。
2.装饰物宝箱开出来的装饰物，放在主城里，可以修饰主城并带来各种BUFF，最多生效有数量限制。
3.放置后的BUFF效果，装饰属性可以在建筑神奇宝箱里面的BUFF分页看到，所有的装饰属性可以在BUFF旁边的图鉴分页点击查看。
4.有关装饰品合成问题：在整体数量生效有限制的情况下，三合一升级更高一级的时候没有直接按照玩家理解的原有属性*3叠加，是一级比一级更高。
5.由于目前游戏内显示的最多是一位小数，所以有一些级别增加了但是没有显示出来，导致玩家认为没有增加，例如从0.2提升到0.24，游戏内目前显示都是0.2。
6.高等级装饰可以分解为低等级装饰。

-------[1]-----New FAQ--------

问题1: 我如何获得装饰物？


回答1: 装饰物可以通过摆放在基地，每周会根据活跃度获得一个免费的永久装饰物，也可以通过购买或者参与活动获取更多的装饰物。

问题2: 我最多能放置多少个装饰物？


回答2: 你最多可以放置20个装饰物，但是每个装饰物的属性都是有限的，并且只有一个建筑能够放置装饰物，所以在放置装饰物时需要谨慎选择。

问题3: 我如何查看我的装饰物的属性？


回答3: 在主城市中，你可以找到"建筑-神奇宝箱"的按钮，点击后会显示你所有已放置的装饰物的属性，还可以通过点击"图鉴"按钮查看所有的装饰物属性。

问题4: 我如何合

## 把一些不完整的FAQ去除

In [654]:
import os
def postprocess_faq(file_content,QA_SEP='====='):
    arr = file_content.split(QA_SEP)
    faqtext = ''
    for item in arr:
        try:
            question, answer = item.strip().split("\n", 1)
            question = question.replace("Question:", "").strip()
            answer = answer.replace("Answer:", "").strip()
            if question != '' and answer != '':
                faqtext+=f"Question: {question}\nAnswer: {answer}\n=====\n"
        except:
            pass
    return faqtext[:-6]


cleaned_new_faq = None
with open(newfaq_filename,"r") as f:
    cleaned_new_faq = postprocess_faq(f.read())
    


## 保存最终的FAQ文件，下一步可以导出再由人工审核

In [658]:
cleaned_filename = 'docs/cleaned_topwar_enrich_faq_0911.faq'
with open(cleaned_filename,"w") as f:
    f.write(cleaned_new_faq)

In [659]:
print(cleaned_new_faq)

Question: 我如何获得装饰物？
Answer: 装饰物可以通过摆放在基地，每周会根据活跃度获得一个免费的永久装饰物，也可以通过购买或者参与活动获取更多的装饰物。
=====
Question: 我最多能放置多少个装饰物？
Answer: 你最多可以放置20个装饰物，但是每个装饰物的属性都是有限的，并且只有一个建筑能够放置装饰物，所以在放置装饰物时需要谨慎选择。
=====
Question: 我如何查看我的装饰物的属性？
Answer: 在主城市中，你可以找到"建筑-神奇宝箱"的按钮，点击后会显示你所有已放置的装饰物的属性，还可以通过点击"图鉴"按钮查看所有的装饰物属性。
=====
Question: 我如何合成更高等级的装饰物？
Answer: 在合成高等级装饰物时，需要将三个同等级的装饰物进行合成，合成后会得到一个更高一级的装饰物，但是合成后不会将原有装饰物的属性直接叠加，而是按照合成后的新属性进行叠加。
=====
Question: 我的游戏等级显示为什么只有0.2而没有显示0.24？
Answer: 由于游戏内显示的最多有一位小数，所以当你从0.2提升到0.24时，游戏内会显示0.2，但是实际上你的游戏等级已经得到了提升。
=====
Question: 我如何分解高等级装饰物？
Answer: 你可以通过消耗一定数量的经验值来分解高等级装饰物，分解后会获得对应的经验值和低等级的装饰物。
=====
Question: 请问专3技能有哪些可以升级的选项？
Answer: 专3技能有多种升级选项，包括攻击力、生命值、防御力、移动速度等。你可以根据自己的需要和战斗策略来选择合适的升级选项。
=====
Question: 我需要多少经验才能将专3技能升级到下一个等级？
Answer: 每次升级专3技能需要一定数量的经验。具体情况可以参考游戏内的指导手册或者咨询游戏内的客服。一般情况下，你需要积累一定数量的经验点数才能将专3技能升级到下一个等级。
=====
Question: 我在游戏里达到了专5级，这意味着什么？
Answer: 专5级代表着你在游戏中的专属技能已经到达了最高水平，对应的技能也变得更加强大可靠。这意味着您在游戏中的表现将更加突出，能够应对更高级别的挑战。此外，专5级也可以为您的角色带来更高的经验和能力加成。
=====
Questi